In [2]:
import os
print(os.getcwd())

/home/broilo/Documents/GitHub/Dataset/UFC-project/data-mining/instagram/Selenium/Conor-McGregor/mcgregormania


In [3]:
path = '/home/broilo/Documents/GitHub/Dataset/UFC-project/data-mining/instagram/Selenium/Conor-McGregor/mcgregormania/'

In [4]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import re
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np

In [5]:
#!pip install selenium

In [6]:
username = 'mcgregormania'
browser = webdriver.Chrome('/usr/bin/chromedriver')
browser.get('https://www.instagram.com/'+username+'/?hl=en')
Pagelength = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [7]:
#Extract links from user profile page
links = ['https://www.instagram.com/p/CD9ouqZBeAe/']
source = browser.page_source
data = bs(source, 'html.parser')
body = data.find('body')
script = body.find('script', text = lambda t: t.startswith('window._sharedData'))
page_json = script.text.split(' = ', 1)[1].rstrip(';')
data = json.loads(page_json)
#try 'script.string' instead of script.text if you get error on index out of range
for link in data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['edges']:
    links.append('https://www.instagram.com'+'/p/'+link['node']['shortcode']+'/')
#try with ['display_url'] instead of ['shortcode'] if you don't get links 

In [8]:
link

{'node': {'__typename': 'GraphVideo',
  'id': '2367764752199064440',
  'shortcode': 'CDb_X9GhBt4',
  'dimensions': {'height': 750, 'width': 750},
  'display_url': 'https://instagram.fpet1-1.fna.fbcdn.net/v/t51.2885-15/e35/116800607_3554784621271113_1629730966897509613_n.jpg?_nc_ht=instagram.fpet1-1.fna.fbcdn.net&_nc_cat=109&_nc_ohc=YoJeYf22wOUAX-_1ZuJ&oh=b7fb8a336f9e8f50ccc89c89f892601f&oe=5F3D1D01',
  'edge_media_to_tagged_user': {'edges': [{'node': {'user': {'full_name': 'Conor McGregor Official',
       'id': '421468021',
       'is_verified': True,
       'profile_pic_url': 'https://instagram.fpet1-1.fna.fbcdn.net/v/t51.2885-19/s150x150/102439603_279373383260127_2922330374362660300_n.jpg?_nc_ht=instagram.fpet1-1.fna.fbcdn.net&_nc_ohc=-jaW5op8cLoAX9xn8Yr&oh=0d1d39175eb60c1f2ac0353dc6ff56c0&oe=5F62D2B6',
       'username': 'thenotoriousmma'},
      'x': 0.0,
      'y': 0.0}}]},
  'fact_check_overall_rating': None,
  'fact_check_information': None,
  'gating_info': None,
  'media_over

In [9]:
result = pd.DataFrame()
for i in range(len(links)):
 try:
     page = urlopen(links[i]).read()
     data = bs(page, 'html.parser')
     body = data.find('body')
     script = body.find('script')
     raw = script.text.strip().replace('window._sharedData =', '').replace(';', '')
     json_data = json.loads(raw)
     posts = json_data['entry_data']['PostPage'][0]['graphql']
     posts= json.dumps(posts)
     posts = json.loads(posts)
     x = pd.DataFrame.from_dict(json_normalize(posts), orient = 'columns') 
     x.columns = x.columns.str.replace('shortcode_media.', '')
     result = result.append(x)
 except:
     np.nan
result = result.drop_duplicates(subset = "shortcode")
result.index = range(len(result.index))

/home/broilo/.local/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  del sys.path[0]


In [10]:
result

,__typename,id,shortcode,dimensions.height,dimensions.width,gating_info,fact_check_overall_rating,fact_check_information,sensitivity_friction_info,media_overlay_info,...,video_url,video_view_count,video_play_count,encoding_status,is_published,product_type,title,video_duration,thumbnail_src,clips_music_attribution_info
0,GraphImage,2377235306794835998,CD9ouqZBeAe,1347,1080,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GraphImage,2376536179039764434,CD7JxA7h3_S,1349,1080,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GraphImage,2375672146308726706,CD4FTrvhX-y,1343,1080,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GraphSidecar,2375051792842339123,CD14QWjhbMz,640,640,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GraphImage,2374037042327131770,CDyRhyfhxZ6,1326,1080,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,GraphImage,2373515006634719533,CDwa1K3hXkt,1340,1080,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,GraphImage,2372225905000393032,CDr1uRahylI,1080,1080,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,GraphImage,2370555319043902554,CDl54DABkxa,1284,1080,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,GraphImage,2369900076576639815,CDjk5A5hYtH,1080,1080,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,GraphImage,2369163846972931623,CDg9fdphP4n,1338,1080,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
import os
import requests
result.index = range(len(result.index))
directory = path
for i in range(len(result)):
    r = requests.get(result['display_url'][i])
    with open(directory+result['shortcode'][i]+".jpg", 'wb') as f:
                    f.write(r.content)

In [12]:
result.shortcode

0     CD9ouqZBeAe
1     CD7JxA7h3_S
2     CD4FTrvhX-y
3     CD14QWjhbMz
4     CDyRhyfhxZ6
5     CDwa1K3hXkt
6     CDr1uRahylI
7     CDl54DABkxa
8     CDjk5A5hYtH
9     CDg9fdphP4n
10    CDcmvlShDAb
11    CDb_X9GhBt4
Name: shortcode, dtype: object